# Example of using simsopt to optimize curve objects

This notebook demonstrates a silly little example of optimizing a curve using simsopt. We start with a curve that has random Fourier coefficients for $R(\phi)$ and $Z(\phi)$, where $(R, \phi, Z)$ are cylindrical coordinates. We minimize the length of the curve, keeping the 0-frequency Fourier mode of $R$ fixed. The result should be a circle.

In [1]:
import numpy as np
from simsopt.geo.curverzfourier import CurveRZFourier
from simsopt.geo.curveobjectives import CurveLength
from simsopt import LeastSquaresProblem
from simsopt import least_squares_serial_solve

Initialize the curve:

In [2]:
nquadrature = 100
nfourier = 4
nfp = 5
curve = CurveRZFourier(nquadrature, nfourier, nfp, True)

Initialize the Fourier coefficients to random numbers. The first coefficient is the average major radius, so set it to a positive number.

In [3]:
x0 = np.random.rand(curve.num_dofs()) - 0.5
x0[0] = 3.0
curve.set_dofs(x0)
print('Initial curve dofs: ', curve.get_dofs())

Initial curve dofs:  [ 3.          0.28502471  0.29757522  0.01220562 -0.31103223  0.02554628
 -0.03307268 -0.17245829 -0.12146519]


Tell the curve object that the first Fourier mode is fixed, whereas all the other dofs are not.

In [4]:
curve.all_fixed(False)
curve.fixed[0] = True

The length objective is a separate object rather than as a function of Curve itself, so we must create this 2nd object. This results in python importing Jax, which usually gives a warning that no GPU/TPU is found - you can ignore this.

In [5]:
obj = CurveLength(curve)
print('Initial curve length: ', obj.J())

Initial curve length:  37.72142781143831


Create a term in the least-squares objective function, $1.0 * (length - 0.0)^2$:

In [6]:
term1 = (obj, 0.0, 1.0)

Put this term into a least-squares problem object:

In [7]:
prob = LeastSquaresProblem([term1])

Simsopt recognizes that derivative information is available:

In [8]:
prob.dofs.grad_avail

True

Let's view the initial global state vector. It should contain all the initial Fourier modes except the first:

In [9]:
prob.x

array([ 0.28502471,  0.29757522,  0.01220562, -0.31103223,  0.02554628,
       -0.03307268, -0.17245829, -0.12146519])

Solve the minimization problem:

In [10]:
least_squares_serial_solve(prob)

Using derivatives
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.1145e+02                                    2.15e+03    
       1              2         3.9325e+02      3.18e+02       5.59e-01       1.19e+03    
       2              4         2.4492e+02      1.48e+02       1.40e-01       4.57e+02    
       3              6         2.1402e+02      3.09e+01       6.99e-02       2.68e+02    
       4              7         1.9785e+02      1.62e+01       6.99e-02       1.61e+02    
       5              8         1.9018e+02      7.67e+00       1.40e-01       2.85e+02    
       6             10         1.8353e+02      6.65e+00       3.50e-02       7.24e+01    
       7             11         1.8176e+02      1.77e+00       3.50e-02       7.76e+01    
       8             12         1.8132e+02      4.38e-01       3.50e-02       1.31e+02    
       9             13         1.8034e+02      9.84e-01       8.74e-03 

Examine properties of the optimum found:

In [11]:
print('At the optimum, x: ', prob.x)
print(' Final curve dofs: ', curve.get_dofs())
print(' Final curve length:    ', obj.J())
print(' Expected final length: ', 2 * np.pi * x0[0])
print(' objective function: ', prob.objective())

At the optimum, x:  [ 1.45167296e-04  2.48260354e-09 -7.23396263e-10  1.34054125e-05
  1.29163174e-05  2.93721927e-10 -1.48232529e-10  2.69514280e-06]
 Final curve dofs:  [ 3.00000000e+00  1.45167296e-04  2.48260354e-09 -7.23396263e-10
  1.34054125e-05  1.29163174e-05  2.93721927e-10 -1.48232529e-10
  2.69514280e-06]
 Final curve length:     18.849556238733285
 Expected final length:  18.84955592153876
 objective function:  355.30577039716894


All the Fourier modes after the first (the 0-frequency mode) have been reduced towards 0. The final curve length is approximately that of the circle with the expected radius.